In [231]:
from nba_api.stats.endpoints import PlayerGameLog, TeamGameLog, LeagueGameLog
from nba_api.stats.static import players
from nba_api.stats.static import teams
import pandas as pd

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 60)

In [232]:
player_name = 'James Harden'
player = players.find_players_by_full_name(player_name)[0]
player_id = player['id']

In [233]:
game_log = PlayerGameLog(player_id=player_id, season="2023-24").get_data_frames()[0]
league_log = LeagueGameLog(season='2023-24').get_data_frames()[0]
game_log

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22023,201935,0022301184,"APR 12, 2024",LAC vs. UTA,L,10,1,4,0.250,0,2,0.000,2,3,0.667,0,0,0,5,0,0,1,0,4,8,1
1,22023,201935,0022301132,"APR 07, 2024",LAC vs. CLE,W,26,6,11,0.545,4,8,0.500,6,6,1.000,0,5,5,5,0,0,2,3,22,-20,1
2,22023,201935,0022301126,"APR 05, 2024",LAC vs. UTA,W,33,4,10,0.400,2,7,0.286,3,3,1.000,0,10,10,15,0,0,5,1,13,19,1
3,22023,201935,0022301114,"APR 04, 2024",LAC vs. DEN,W,38,6,23,0.261,2,13,0.154,6,7,0.857,1,5,6,8,1,0,1,0,20,4,1
4,22023,201935,0022301101,"APR 02, 2024",LAC @ SAC,L,31,1,7,0.143,1,5,0.200,3,4,0.750,1,4,5,8,1,0,5,0,6,-13,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,22023,201935,0022300024,"NOV 14, 2023",LAC @ DEN,L,35,7,15,0.467,3,8,0.375,4,4,1.000,0,2,2,4,0,0,1,3,21,-3,1
68,22023,201935,0022300179,"NOV 12, 2023",LAC vs. MEM,L,29,4,12,0.333,1,7,0.143,2,2,1.000,2,2,4,3,1,1,1,0,11,-28,1
69,22023,201935,0022300014,"NOV 10, 2023",LAC @ DAL,L,29,3,6,0.500,2,3,0.667,6,6,1.000,0,5,5,3,2,0,2,0,14,-6,1
70,22023,201935,0022300160,"NOV 08, 2023",LAC @ BKN,L,36,4,9,0.444,2,5,0.400,2,2,1.000,0,8,8,5,2,0,5,0,12,-15,1


In [234]:
final_df = pd.DataFrame(game_log[['GAME_DATE', 'WL', 'MIN', 'FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PTS','PLUS_MINUS']])
league_df = pd.DataFrame(league_log[['GAME_DATE', 'WL','MIN','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PTS'	,'PLUS_MINUS']])

In [235]:
final_df['GAME_DATE'] = pd.to_datetime(final_df['GAME_DATE'], format='mixed')
league_df['GAME_DATE'] = pd.to_datetime(league_df['GAME_DATE'], format='mixed')

In [236]:
final_df['HOME'] = game_log['MATCHUP'].apply(lambda x: 1 if 'vs. ' in x else 0)
final_df['TEAM'] = game_log['MATCHUP'].apply(lambda x: x.split()[0])
final_df['OPPONENT'] = game_log['MATCHUP'].apply(lambda x: x.split()[-1])

league_df['HOME'] = league_log['MATCHUP'].apply(lambda x: 1 if 'vs. ' in x else 0)
league_df['TEAM'] = league_log['MATCHUP'].apply(lambda x: x.split()[0])
league_df['OPPONENT'] = league_log['MATCHUP'].apply(lambda x: x.split()[-1])

In [237]:
merged1 = final_df.merge(league_df, how = 'left', left_on = ['GAME_DATE', 'TEAM', 'OPPONENT'], right_on=['GAME_DATE', 'TEAM', 'OPPONENT'])
merged2 = merged1.merge(league_df, how = 'left', left_on = ['GAME_DATE', 'OPPONENT'], right_on=['GAME_DATE', 'TEAM'])

In [238]:
merged2['date_shifted'] = merged2['GAME_DATE'].shift(1)
merged2['days_rest'] = merged2.date_shifted - merged2.GAME_DATE
merged2['days_rest'] = merged2['days_rest'].dt.days

AttributeError: 'DataFrame' object has no attribute 'date_shifted'

In [ ]:
merged2['rolling_avg'] = merged2['PTS_x'].rolling(window=5, min_periods=1).mean().values
merged2['rolling_3pa_avg'] = merged2['FG3A_x'].rolling(window=5, min_periods=1).mean().values
merged2['min_rolling'] = merged2['MIN_x'].rolling(window=5, min_periods=1).mean().values
merged2['rolling_team_fta'] = merged2['FTA_y'].rolling(window=5, min_periods=1).mean().values
merged2['rolling_opp_pts'] = merged2['PTS'].rolling(window=5, min_periods=1).mean().values
merged2

,GAME_DATE,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,HOME,TEAM,OPPONENT,date_shifted,days_rest,rolling_avg,rolling_3pa_avg,min_rolling
0,2024-04-12,L,10,1,4,0.250,0,2,0.000,2,3,0.667,0,0,0,5,0,0,1,0,4,8,1,LAC,UTA,NaT,NaN,4.00,2.000000,10.00
1,2024-04-07,W,26,6,11,0.545,4,8,0.500,6,6,1.000,0,5,5,5,0,0,2,3,22,-20,1,LAC,CLE,2024-04-12,5.0,13.00,5.000000,18.00
2,2024-04-05,W,33,4,10,0.400,2,7,0.286,3,3,1.000,0,10,10,15,0,0,5,1,13,19,1,LAC,UTA,2024-04-07,2.0,13.00,5.666667,23.00
3,2024-04-04,W,38,6,23,0.261,2,13,0.154,6,7,0.857,1,5,6,8,1,0,1,0,20,4,1,LAC,DEN,2024-04-05,1.0,14.75,7.500000,26.75
4,2024-04-02,L,31,1,7,0.143,1,5,0.200,3,4,0.750,1,4,5,8,1,0,5,0,6,-13,0,LAC,SAC,2024-04-04,2.0,13.00,7.000000,27.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2023-11-14,L,35,7,15,0.467,3,8,0.375,4,4,1.000,0,2,2,4,0,0,1,3,21,-3,0,LAC,DEN,2023-11-17,3.0,16.40,6.000000,34.00
68,2023-11-12,L,29,4,12,0.333,1,7,0.143,2,2,1.000,2,2,4,3,1,1,1,0,11,-28,1,LAC,MEM,2023-11-14,2.0,17.00,6.200000,32.40
69,2023-11-10,L,29,3,6,0.500,2,3,0.667,6,6,1.000,0,5,5,3,2,0,2,0,14,-6,0,LAC,DAL,2023-11-12,2.0,16.60,6.000000,31.00
70,2023-11-08,L,36,4,9,0.444,2,5,0.400,2,2,1.000,0,8,8,5,2,0,5,0,12,-15,0,LAC,BKN,2023-11-10,2.0,16.40,5.400000,32.60
